# Goodinfo 策略回測

- 查詢策略
- 記錄日期並分類存入 DataBase

In [1]:
from stock_fuction import *
from datetime import datetime, timedelta
import requests

In [17]:
date_str = "20250212"
insert_stock_to_db(date_str)

In [13]:
def notify_discord_webhook(msg):
	url = 'https://discord.com/api/webhooks/1337682892565774418/UY9lg-YV47mYTjKBO1POOiai72raTrfOpfL7svkC1khEwpszToNN5b13OAK0jZIDc4oK'
	headers = {"Content-Type": "application/json"}
	data = {"content": msg, "username": "newmanBot"}
	res = requests.post(url, headers = headers, json = data) 
	if res.status_code in (200, 204):
		return True
	else:
		print(f"Request failed with response: {res.status_code}-{res.text}")
		return False

In [4]:
notify_discord_webhook("測試")

Request fulfilled with response: 


In [ ]:
cnxn, cursor = connet_to_db()

send_text = f"低本益比 + 低股價淨值比 + 穩定股利 + 交易量大於1000張 (Code, 名稱, 產業, 上市/上櫃, 收盤價, 交易量, PE, 股價淨值比, 殖利率)：\n"

send_sql = """
SELECT STRING_AGG('(' + Code + ', ' + Name + ', ' + Industry + ', ' + Market + ', ' +
CAST(ClosingPrice AS NVARCHAR) + ', ' + CAST([Transaction] AS NVARCHAR) + ', ' + CAST(PE AS NVARCHAR) + ', ' + 
CAST(股價淨值比 AS NVARCHAR) + ', ' + CAST(殖利率 AS NVARCHAR) + ')', ',\n') WITHIN GROUP (ORDER BY PE ASC, 殖利率 DESC) AS result
FROM vd_StockDayInfo
WHERE Date = (SELECT TOP 1 Date FROM vd_StockDayInfo ORDER BY Date DESC) 
        AND PE < 15 
        AND 殖利率 > 5 
        AND [Transaction] > 1000
"""

cursor.execute(send_sql)
result = cursor.fetchone()

send_text += result[0]

notify_discord_webhook(send_text)

close_db_connection(cnxn, cursor)